In [10]:
# Import necessary packages
import sys

import pandas as pd
import numpy as np

from priority_dict import priority_dict
from pptree import print_tree

In [3]:
def read_file(file_name):
    """
    Creates an adjacency matrix from the csv file
    """
    df = pd.read_csv(file_name, header=None)
    
    # Dictionary to store cities' indices
    vertices_id = {}
    cnt = 0
    
    # Iterate all rows and assign an unique id to each city
    for _, row in df.iterrows():   
        for city in [row[0], row[1]]:
            if city not in vertices_id:
                vertices_id[city] = cnt
                cnt += 1
    num_cities = len(vertices_id)
    
    # Create an adjacency matrix of shape (num_vertices, num_vertices)
    adj_matrix = np.zeros((num_cities, num_cities))
    # Iterate the rows again to build the adj matrix
    for _, row in df.iterrows():
        idx_1 = vertices_id[row[0]]
        idx_2 = vertices_id[row[1]]
        # Add the connection to adjacency matrix (both directions)
        adj_matrix[idx_1][idx_2] = row[2]
        adj_matrix[idx_2][idx_1] = row[2]
        
    return adj_matrix, vertices_id

In [4]:
class Graph:
    def __init__(self, adj_matrix):
        self.adj_matrix = adj_matrix
        
class Tree:
    def __init__(self, val, parent=None):
        self.val = val
        self.children = []
        self.set_parent(parent)
            
    def set_parent(self, parent):
        self.parent = parent
        if parent:
            self.parent.children.append(self)
    
    def __str__(self):
        return self.val

In [5]:
def degree(G, v):
        """
        Returns the degree of a given vertex
        """
        return len(self.neighbors(v))
    
def neighbors(G, v):
        """
        Returns indices of v's neighbors
        """
        row = G.adj_matrix[v]
        return np.where(row != 0)[0]

def dfs_helper(G, current, parent):
        # Scan through the neighbors of the current vertex
        for neighbor in neighbors(G, current):
            # Check if it is already visited or not
            if neighbor not in parent:
                # We visit this vertex from "current", so mark it as the parent 
                parent[neighbor] = current
                # Recur!
                dfs_helper(G, neighbor, parent)
        
def dfs(G, start):
    """
    Perform DFS from the given starting vertex
    """
    # Store parent information
    parent = {start: None}

    # Call the recursive helper function, real work starts here
    dfs_helper(G, start, parent)
    return parent 

def components(G):
    """
    Returns the list of connected components in the graph
    """
    big_parent = {}
    component_list = []

    # Perform DFS on every vertices, one completion of a DFS corresponds to one connnected component
    for v in range(G.adj_matrix.shape[0]):
        if v not in big_parent:
            parent = dfs(G, v)
            component_list.append(parent)
            big_parent.update(parent)

    return component_list

def path(G, u, v):
        """
        Returns the path between two vertices if it exists
        """
        # Perform DFS from u
        parent = dfs(G, u)
        
        # If a path from u to v exists, backtrack the path from vertex 2 using information stored in parent
        if v in parent:
            current = v
            p = []
            while (current is not None):
                p.append(current)
                current = parent[current]
            p.reverse()
            return p
        else:
            return None

In [22]:
def grow_edge(G, current, parent, T):
    global id_2_name
    for v in neighbors(G, current):
        if v not in parent:
            parent[v] = current
            new_subtree = Tree(id_2_name[v], T)
            grow_edge(G, v, parent, new_subtree)

def spanning_tree(G, start):
    """
    Returns a spanning tree rooted at start, using DFS
    """
    global id_2_name
    T = Tree(id_2_name[start])
    parent = {start: None}
    grow_edge(G, start, parent, T)
    
    return T

def prim(G):
    global id_2_name
    
    V = list(range(G.adj_matrix.shape[0]))
    
    parent = {0: None}
    C = priority_dict()
    for v in V:
        C[v] = sys.maxsize
    C[0] = 0
    
    while len(C) != 0:
        v = C.pop_smallest()
        for w in neighbors(G, v):
            if (w in C) and (C[w] > G.adj_matrix[v][w]):
                C[w] = G.adj_matrix[v][w]
                parent[w] = v
    
    tree_nodes = {}
    for v in V:
        tree_nodes[v] = Tree(id_2_name[v])
        
    for v in parent:
        if parent[v] is not None:
            tree_nodes[v].set_parent(tree_nodes[parent[v]])
    return tree_nodes[0]

In [23]:
def preorder(T):
    print(T.val)
    for child in T.children:
        preorder(child)

In [27]:
adj_matrix, name_2_id = read_file("small_graph.csv")
id_2_name = {v: k for k, v in name_2_id.items()}
G = Graph(adj_matrix)

In [28]:
mst = prim(G)
print_tree(mst)

 A┐
  └E┐
    └D┐
      └B┐
        └C┐
          └F


In [21]:
mst.children

[<__main__.Tree at 0x7f8856d6c128>, <__main__.Tree at 0x7f8856d6c198>]